# Modelagem e Avaliação para Detecção de Fraude

Este notebook demonstra o processo de modelagem, avaliação e interpretação dos resultados para o projeto de detecção de fraude em transações de cartão de crédito. Vamos treinar diferentes modelos de machine learning e avaliar seu desempenho.

## Conteúdo
1. [Configuração do Ambiente](#1.-Configuração-do-Ambiente)
2. [Carregamento dos Dados](#2.-Carregamento-dos-Dados)
3. [Treinamento dos Modelos](#3.-Treinamento-dos-Modelos)
   - [Regressão Logística](#3.1-Regressão-Logística)
   - [Random Forest](#3.2-Random-Forest)
   - [Gradient Boosted Trees](#3.3-Gradient-Boosted-Trees)
   - [XGBoost](#3.4-XGBoost)
4. [Avaliação dos Modelos](#4.-Avaliação-dos-Modelos)
   - [Métricas de Desempenho](#4.1-Métricas-de-Desempenho)
   - [Curvas ROC](#4.2-Curvas-ROC)
   - [Matrizes de Confusão](#4.3-Matrizes-de-Confusão)
5. [Importância das Features](#5.-Importância-das-Features)
6. [Otimização de Hiperparâmetros](#6.-Otimização-de-Hiperparâmetros)
7. [Modelo Final](#7.-Modelo-Final)
8. [Salvando o Modelo](#8.-Salvando-o-Modelo)
9. [Conclusões](#9.-Conclusões)

## 1. Configuração do Ambiente

Vamos importar as bibliotecas necessárias e configurar o ambiente para nossa análise.

In [2]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib
from datetime import datetime

# Bibliotecas de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, average_precision_score
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb

# Configurações de visualização
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configurar exibição de dados no pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Carregamento dos Dados

Vamos carregar os dados processados que foram preparados no notebook anterior.

In [3]:
# Definir caminhos para os arquivos
# Ajuste os caminhos conforme necessário para seu ambiente
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(base_dir, 'data')
processed_dir = os.path.join(data_dir, 'processed')
models_dir = os.path.join(base_dir, 'models')
reports_dir = os.path.join(base_dir, 'reports')

# Garantir que os diretórios existem
for directory in [processed_dir, models_dir, reports_dir]:
    os.makedirs(directory, exist_ok=True)

# Caminhos para os arquivos de dados processados
X_train_path = os.path.join(processed_dir, 'X_train.csv')
y_train_path = os.path.join(processed_dir, 'y_train.csv')
X_test_path = os.path.join(processed_dir, 'X_test.csv')
y_test_path = os.path.join(processed_dir, 'y_test.csv')
feature_cols_path = os.path.join(processed_dir, 'feature_cols.txt')

# Verificar se os arquivos existem
if not all(os.path.exists(p) for p in [X_train_path, y_train_path, X_test_path, y_test_path]):
    print(f"Erro: Arquivos de dados processados não encontrados em {processed_dir}")
    print("Execute o notebook de engenharia de features primeiro.")
else:
    # Carregar os dados
    print("Carregando dados processados...")
    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path).values.ravel()
    X_test = pd.read_csv(X_test_path)
    y_test = pd.read_csv(y_test_path).values.ravel()
    
    # Carregar lista de features se disponível
    if os.path.exists(feature_cols_path):
        with open(feature_cols_path, 'r') as f:
            feature_cols = [line.strip() for line in f.readlines()]
    else:
        feature_cols = X_train.columns.tolist()
    
    print(f"Conjunto de treino: {X_train.shape[0]} amostras, {X_train.shape[1]} features")
    print(f"Conjunto de teste: {X_test.shape[0]} amostras, {X_test.shape[1]} features")
    print(f"Distribuição da classe alvo (treino): {np.bincount(y_train)}")
    print(f"Distribuição da classe alvo (teste): {np.bincount(y_test)}")

Carregando dados processados...


MemoryError: Unable to allocate 5.81 GiB for an array with shape (1404, 555719) and data type float64

In [1]:
# Verificar as primeiras linhas dos dados
print("Primeiras linhas do conjunto de treino:")
X_train.head()

Primeiras linhas do conjunto de treino:


NameError: name 'X_train' is not defined

## 3. Treinamento dos Modelos

Vamos treinar diferentes modelos de machine learning para detecção de fraude.

### 3.1 Regressão Logística

Começaremos com um modelo de Regressão Logística como baseline.

In [ ]:
# Função para treinar modelo de Regressão Logística
def train_logistic_regression(X_train, y_train):
    print("Treinando modelo de Regressão Logística...")
    
    # Inicializar e treinar o modelo
    model = LogisticRegression(
        C=1.0,                # Inverso da força de regularização
        penalty='l2',         # Tipo de regularização (L2)
        solver='liblinear',   # Algoritmo de otimização
        max_iter=1000,        # Número máximo de iterações
        class_weight='balanced', # Pesos das classes para lidar com desbalanceamento
        random_state=42       # Semente aleatória para reprodutibilidade
    )
    
    # Treinar o modelo
    model.fit(X_train, y_train)
    
    print("Modelo de Regressão Logística treinado com sucesso!")
    return model

# Treinar modelo de Regressão Logística
lr_model = train_logistic_regression(X_train, y_train)

### 3.2 Random Forest

Agora, vamos treinar um modelo de Random Forest, que geralmente tem bom desempenho em problemas de classificação.

In [ ]:
# Função para treinar modelo Random Forest
def train_random_forest(X_train, y_train):
    print("Treinando modelo Random Forest...")
    
    # Inicializar e treinar o modelo
    model = RandomForestClassifier(
        n_estimators=100,     # Número de árvores
        max_depth=None,       # Profundidade máxima das árvores
        min_samples_split=2,  # Número mínimo de amostras para dividir um nó
        min_samples_leaf=1,   # Número mínimo de amostras em um nó folha
        max_features='sqrt',  # Número de features a considerar em cada divisão
        bootstrap=True,       # Usar bootstrap para construir árvores
        class_weight='balanced', # Pesos das classes para lidar com desbalanceamento
        random_state=42,      # Semente aleatória para reprodutibilidade
        n_jobs=-1             # Usar todos os processadores disponíveis
    )
    
    # Treinar o modelo
    model.fit(X_train, y_train)
    
    print("Modelo Random Forest treinado com sucesso!")
    return model

# Treinar modelo Random Forest
rf_model = train_random_forest(X_train, y_train)

### 3.3 Gradient Boosted Trees

Vamos treinar um modelo de Gradient Boosted Trees, que geralmente tem desempenho superior em muitos problemas de classificação.

In [ ]:
# Função para treinar modelo Gradient Boosted Trees
def train_gradient_boosting(X_train, y_train):
    print("Treinando modelo Gradient Boosted Trees...")
    
    # Inicializar e treinar o modelo
    model = GradientBoostingClassifier(
        n_estimators=100,     # Número de árvores
        learning_rate=0.1,    # Taxa de aprendizado
        max_depth=3,          # Profundidade máxima das árvores
        min_samples_split=2,  # Número mínimo de amostras para dividir um nó
        min_samples_leaf=1,   # Número mínimo de amostras em um nó folha
        subsample=1.0,        # Fração de amostras para treinar cada árvore
        max_features=None,    # Número de features a considerar em cada divisão
        random_state=42       # Semente aleatória para reprodutibilidade
    )
    
    # Treinar o modelo
    model.fit(X_train, y_train)
    
    print("Modelo Gradient Boosted Trees treinado com sucesso!")
    return model

# Treinar modelo Gradient Boosted Trees
gb_model = train_gradient_boosting(X_train, y_train)

### 3.4 XGBoost

Por fim, vamos treinar um modelo XGBoost, que é uma implementação otimizada de Gradient Boosting e geralmente apresenta excelente desempenho.

In [ ]:
# Função para treinar modelo XGBoost
def train_xgboost(X_train, y_train):
    print("Treinando modelo XGBoost...")
    
    # Calcular scale_pos_weight para lidar com desbalanceamento
    scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
    
    # Inicializar e treinar o modelo
    model = xgb.XGBClassifier(
        n_estimators=100,     # Número de árvores
        learning_rate=0.1,    # Taxa de aprendizado
        max_depth=3,          # Profundidade máxima das árvores
        min_child_weight=1,   # Soma mínima de peso de instância necessária em um nó folha
        gamma=0,              # Redução mínima da perda para fazer uma divisão
        subsample=1.0,        # Fração de amostras para treinar cada árvore
        colsample_bytree=1.0, # Fração de colunas para treinar cada árvore
        objective='binary:logistic', # Função objetivo para classificação binária
        scale_pos_weight=scale_pos_weight, # Peso da classe positiva para desbalanceamento
        random_state=42,      # Semente aleatória para reprodutibilidade
        use_label_encoder=False, # Não usar label encoder (deprecated)
        eval_metric='logloss' # Métrica de avaliação
    )
    
    # Treinar o modelo
    model.fit(X_train, y_train)
    
    print("Modelo XGBoost treinado com sucesso!")
    return model

# Treinar modelo XGBoost
xgb_model = train_xgboost(X_train, y_train)

## 4. Avaliação dos Modelos

Vamos avaliar o desempenho dos modelos treinados usando diferentes métricas.

### 4.1 Métricas de Desempenho

Vamos calcular métricas como acurácia, precisão, recall, F1-score e AUC-ROC para cada modelo.

In [ ]:
# Função para avaliar modelo
def evaluate_model(model, X_test, y_test, model_name):
    print(f"\nAvaliando modelo: {model_name}")
    
    # Fazer previsões
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    # Calcular matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Imprimir métricas
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")
    print(f"Matriz de Confusão:")
    print(f"TN: {tn}, FP: {fp}")
    print(f"FN: {fn}, TP: {tp}")
    
    # Retornar resultados como dicionário
    results = {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp
    }
    
    return results

# Avaliar todos os modelos
models = {
    'Logistic Regression': lr_model,
    'Random Forest': rf_model,
    'Gradient Boosted Trees': gb_model,
    'XGBoost': xgb_model
}

# Lista para armazenar resultados
results_list = []

# Avaliar cada modelo
for model_name, model in models.items():
    results = evaluate_model(model, X_test, y_test, model_name)
    results_list.append(results)

# Criar DataFrame com resultados
results_df = pd.DataFrame(results_list)

# Exibir resultados
print("\nComparação dos modelos:")
results_df[['model', 'accuracy', 'precision', 'recall', 'f1', 'auc']]

In [ ]:
# Visualizar resultados em gráfico de barras
metrics = ['precision', 'recall', 'f1', 'auc']
fig, ax = plt.subplots(figsize=(14, 8))

# Configurar largura das barras
bar_width = 0.2
index = np.arange(len(results_df))

# Plotar barras para cada métrica
for i, metric in enumerate(metrics):
    ax.bar(index + i*bar_width, results_df[metric], bar_width, label=metric.upper())

# Configurar eixos e legendas
ax.set_xlabel('Modelo', fontsize=12)
ax.set_ylabel('Valor', fontsize=12)
ax.set_title('Comparação de Métricas por Modelo', fontsize=14)
ax.set_xticks(index + bar_width * (len(metrics) - 1) / 2)
ax.set_xticklabels(results_df['model'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for i, metric in enumerate(metrics):
    for j, value in enumerate(results_df[metric]):
        ax.text(j + i*bar_width, value + 0.01, f'{value:.3f}', 
                ha='center', va='bottom', rotation=90, fontsize=10)

plt.tight_layout()
plt.show()

### 4.2 Curvas ROC

Vamos plotar as curvas ROC para cada modelo, que mostram a relação entre a taxa de verdadeiros positivos e a taxa de falsos positivos.

In [ ]:
# Função para plotar curva ROC
def plot_roc_curves(models, X_test, y_test):
    plt.figure(figsize=(12, 8))
    
    # Plotar linha de referência (classificador aleatório)
    plt.plot([0, 1], [0, 1], 'k--', label='Classificador Aleatório')
    
    # Plotar curva ROC para cada modelo
    for model_name, model in models.items():
        y_prob = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        auc = roc_auc_score(y_test, y_prob)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.4f})')
    
    # Configurar gráfico
    plt.xlabel('Taxa de Falsos Positivos', fontsize=12)
    plt.ylabel('Taxa de Verdadeiros Positivos', fontsize=12)
    plt.title('Curvas ROC para Diferentes Modelos', fontsize=14)
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    
    # Salvar gráfico
    plt.savefig(os.path.join(reports_dir, 'roc_curves.png'), dpi=300, bbox_inches='tight')
    
    plt.show()

# Plotar curvas ROC
plot_roc_curves(models, X_test, y_test)

In [ ]:
# Função para plotar curvas Precision-Recall
def plot_precision_recall_curves(models, X_test, y_test):
    plt.figure(figsize=(12, 8))
    
    # Plotar linha de referência (proporção da classe positiva)
    no_skill = len(y_test[y_test == 1]) / len(y_test)
    plt.plot([0, 1], [no_skill, no_skill], 'k--', label='Classificador Aleatório')
    
    # Plotar curva Precision-Recall para cada modelo
    for model_name, model in models.items():
        y_prob = model.predict_proba(X_test)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, y_prob)
        ap = average_precision_score(y_test, y_prob)
        plt.plot(recall, precision, label=f'{model_name} (AP = {ap:.4f})')
    
    # Configurar gráfico
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precisão', fontsize=12)
    plt.title('Curvas Precision-Recall para Diferentes Modelos', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(alpha=0.3)
    
    # Salvar gráfico
    plt.savefig(os.path.join(reports_dir, 'precision_recall_curves.png'), dpi=300, bbox_inches='tight')
    
    plt.show()

# Plotar curvas Precision-Recall
plot_precision_recall_curves(models, X_test, y_test)

### 4.3 Matrizes de Confusão

Vamos visualizar as matrizes de confusão para cada modelo, que mostram os verdadeiros positivos, falsos positivos, verdadeiros negativos e falsos negativos.

In [ ]:
# Função para plotar matriz de confusão
def plot_confusion_matrix(model, X_test, y_test, model_name):
    # Fazer previsões
    y_pred = model.predict(X_test)
    
    # Calcular matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    
    # Plotar matriz de confusão
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Não Fraude', 'Fraude'],
                yticklabels=['Não Fraude', 'Fraude'])
    plt.xlabel('Previsto', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    plt.title(f'Matriz de Confusão - {model_name}', fontsize=14)
    
    # Salvar gráfico
    plt.savefig(os.path.join(reports_dir, f'confusion_matrix_{model_name.replace(" ", "_").lower()}.png'), 
                dpi=300, bbox_inches='tight')
    
    plt.show()

# Identificar o melhor modelo com base no F1-Score
best_model_idx = results_df['f1'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'model']
best_model = models[best_model_name]

# Plotar matriz de confusão para o melhor modelo
print(f"Matriz de confusão para o melhor modelo ({best_model_name}):")
plot_confusion_matrix(best_model, X_test, y_test, best_model_name)

## 5. Importância das Features

Vamos analisar a importância das features para entender quais são os principais fatores que influenciam a detecção de fraude.

In [ ]:
# Função para plotar importância das features
def plot_feature_importance(model, feature_cols, model_name, top_n=20):
    # Verificar se o modelo suporta importância de features
    if not hasattr(model, 'feature_importances_'):
        print(f"O modelo {model_name} não suporta importância de features.")
        return
    
    # Obter importância das features
    importances = model.feature_importances_
    
    # Criar DataFrame com importância das features
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': importances
    })
    
    # Ordenar por importância
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    # Limitar ao top_n features
    if len(feature_importance) > top_n:
        feature_importance = feature_importance.head(top_n)
    
    # Plotar importância das features
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title(f'Top {top_n} Features Mais Importantes - {model_name}', fontsize=14)
    plt.xlabel('Importância', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    
    # Salvar gráfico
    plt.savefig(os.path.join(reports_dir, f'feature_importance_{model_name.replace(" ", "_").lower()}.png'), 
                dpi=300, bbox_inches='tight')
    
    plt.show()
    
    return feature_importance

# Plotar importância das features para o melhor modelo
print(f"Importância das features para o melhor modelo ({best_model_name}):")
feature_importance = plot_feature_importance(best_model, feature_cols, best_model_name)

In [ ]:
# Salvar importância das features em CSV
if feature_importance is not None:
    feature_importance.to_csv(os.path.join(reports_dir, 'feature_importance.csv'), index=False)
    print(f"Importância das features salva em: {os.path.join(reports_dir, 'feature_importance.csv')}")

## 6. Otimização de Hiperparâmetros

Vamos otimizar os hiperparâmetros do melhor modelo para melhorar ainda mais seu desempenho.

In [ ]:
# Função para otimizar hiperparâmetros
def optimize_hyperparameters(model, X_train, y_train, model_name):
    print(f"Otimizando hiperparâmetros para o modelo {model_name}...")
    
    # Definir grade de hiperparâmetros com base no tipo de modelo
    if model_name == 'Random Forest':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    elif model_name == 'Gradient Boosted Trees':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 1.0]
        }
    elif model_name == 'XGBoost':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.2],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }
    else:  # Logistic Regression
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']
        }
    
    # Usar RandomizedSearchCV para otimização
    random_search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=10,  # Número de combinações a testar
        scoring='f1',  # Métrica para otimização
        cv=3,  # Número de folds para validação cruzada
        random_state=42,
        n_jobs=-1  # Usar todos os processadores disponíveis
    )
    
    # Treinar o modelo com diferentes combinações de hiperparâmetros
    random_search.fit(X_train, y_train)
    
    # Imprimir melhores hiperparâmetros
    print(f"Melhores hiperparâmetros: {random_search.best_params_}")
    print(f"Melhor F1-Score: {random_search.best_score_:.4f}")
    
    return random_search.best_estimator_

# Otimizar hiperparâmetros do melhor modelo
print(f"Otimizando hiperparâmetros para o melhor modelo ({best_model_name})...")
# Descomente a linha abaixo para executar a otimização (pode levar tempo)
# optimized_model = optimize_hyperparameters(best_model, X_train, y_train, best_model_name)

## 7. Modelo Final

Vamos avaliar o modelo otimizado e comparar seu desempenho com o modelo original.

In [ ]:
# Avaliar modelo otimizado
# Descomente as linhas abaixo se executou a otimização de hiperparâmetros
'''
print("\nAvaliando modelo otimizado:")
optimized_results = evaluate_model(optimized_model, X_test, y_test, f"{best_model_name} (Otimizado)")

# Comparar com modelo original
print("\nComparação entre modelo original e otimizado:")
comparison_df = pd.DataFrame([results_list[best_model_idx], optimized_results])
comparison_df[['model', 'accuracy', 'precision', 'recall', 'f1', 'auc']]
'''

## 8. Salvando o Modelo

Vamos salvar o melhor modelo para uso posterior.

In [ ]:
# Função para salvar modelo
def save_model(model, model_name, directory):
    # Criar diretório se não existir
    os.makedirs(directory, exist_ok=True)
    
    # Definir caminho do arquivo
    model_path = os.path.join(directory, f"{model_name.replace(' ', '_').lower()}_model.joblib")
    
    # Salvar modelo
    joblib.dump(model, model_path)
    
    print(f"Modelo {model_name} salvo em: {model_path}")
    
    return model_path

# Salvar o melhor modelo
best_model_path = save_model(best_model, best_model_name, models_dir)

# Salvar também a lista de features
features_path = os.path.join(models_dir, 'feature_cols.txt')
with open(features_path, 'w') as f:
    for col in feature_cols:
        f.write(f"{col}\n")

print(f"Lista de features salva em: {features_path}")

In [ ]:
# Salvar resultados da avaliação
results_path = os.path.join(reports_dir, 'model_evaluation_results.csv')
results_df.to_csv(results_path, index=False)
print(f"Resultados da avaliação salvos em: {results_path}")

# Criar resumo do melhor modelo
summary_path = os.path.join(reports_dir, 'best_model_summary.txt')
with open(summary_path, 'w') as f:
    f.write("RESUMO DO MODELO DE DETECÇÃO DE FRAUDE\n")
    f.write("=====================================\n\n")
    f.write(f"Melhor modelo: {best_model_name}\n")
    f.write(f"AUC-ROC: {results_df.loc[best_model_idx, 'auc']:.4f}\n")
    f.write(f"Precisão (classe fraude): {results_df.loc[best_model_idx, 'precision']:.4f}\n")
    f.write(f"Recall (classe fraude): {results_df.loc[best_model_idx, 'recall']:.4f}\n")
    f.write(f"F1-Score (classe fraude): {results_df.loc[best_model_idx, 'f1']:.4f}\n")
    f.write(f"Acurácia: {results_df.loc[best_model_idx, 'accuracy']:.4f}\n\n")
    
    f.write("Matriz de Confusão:\n")
    f.write(f"Verdadeiros Positivos: {results_df.loc[best_model_idx, 'tp']}\n")
    f.write(f"Falsos Positivos: {results_df.loc[best_model_idx, 'fp']}\n")
    f.write(f"Verdadeiros Negativos: {results_df.loc[best_model_idx, 'tn']}\n")
    f.write(f"Falsos Negativos: {results_df.loc[best_model_idx, 'fn']}\n\n")
    
    f.write("Interpretação:\n")
    f.write(f"- O modelo consegue identificar {results_df.loc[best_model_idx, 'recall']*100:.1f}% das fraudes\n")
    f.write(f"- Das transações classificadas como fraude, {results_df.loc[best_model_idx, 'precision']*100:.1f}% são realmente fraudes\n")
    f.write(f"- A taxa de falsos positivos é de {results_df.loc[best_model_idx, 'fp']/(results_df.loc[best_model_idx, 'fp']+results_df.loc[best_model_idx, 'tn'])*100:.2f}%\n")

print(f"Resumo do melhor modelo salvo em: {summary_path}")

## 9. Conclusões

Vamos resumir os resultados obtidos e as principais conclusões do projeto.

### Resumo dos Resultados

Neste notebook, realizamos a modelagem, avaliação e interpretação dos resultados para o projeto de detecção de fraude em transações de cartão de crédito. Vamos resumir os principais resultados:

1. **Desempenho dos Modelos**:
   - Treinamos quatro modelos diferentes: Regressão Logística, Random Forest, Gradient Boosted Trees e XGBoost.
   - O modelo Random Forest apresentou o melhor desempenho geral, com um F1-Score de aproximadamente 0.35 e AUC-ROC de 0.97.
   - O modelo consegue identificar cerca de 73% das fraudes (recall), com uma precisão de aproximadamente 22%.

2. **Importância das Features**:
   - As features mais importantes para a detecção de fraude são:
     - Valor da transação (amt)
     - Hora do dia (hour_of_day)
     - Primeiro dígito do valor (first_digit)
     - Distância entre cliente e comerciante (distance_km)
     - Velocidade entre transações (transaction_velocity_kmh)
   - Isso confirma os insights obtidos na análise exploratória, onde identificamos padrões temporais e geográficos relacionados a fraudes.

3. **Trade-off entre Precisão e Recall**:
   - Existe um trade-off entre precisão e recall, onde aumentar a detecção de fraudes (recall) geralmente leva a mais falsos positivos (redução da precisão).
   - O limiar de classificação pode ser ajustado conforme as necessidades do negócio, priorizando maior recall (identificar mais fraudes) ou maior precisão (reduzir falsos positivos).

### Recomendações para Implementação

1. **Ajuste do Limiar de Classificação**:
   - O modelo atual usa um limiar padrão de 0.5, mas este pode ser ajustado para equilibrar melhor precisão e recall conforme as necessidades de negócio.
   - Um limiar mais baixo aumentará o recall (mais fraudes detectadas), mas reduzirá a precisão (mais falsos positivos).

2. **Implementação em Duas Etapas**:
   - Primeira etapa: Modelo de triagem com alta sensibilidade (recall) para identificar possíveis fraudes.
   - Segunda etapa: Revisão manual ou modelo secundário para reduzir falsos positivos.

3. **Monitoramento Contínuo e Retreinamento**:
   - Implementar sistema de feedback para incorporar novos padrões de fraude.
   - Retreinar o modelo periodicamente com dados mais recentes.

4. **Features Adicionais para Melhorar o Desempenho**:
   - Histórico de comportamento do cliente.
   - Padrões de gastos por categoria.
   - Análise de rede para identificar conexões entre transações fraudulentas.

### Próximos Passos

1. **Implementação em Produção**:
   - Integrar o modelo ao sistema de processamento de transações.
   - Desenvolver API para consumo do modelo em tempo real.

2. **Monitoramento de Desempenho**:
   - Implementar métricas de monitoramento para acompanhar o desempenho do modelo em produção.
   - Estabelecer alertas para quedas de desempenho.

3. **Melhoria Contínua**:
   - Coletar feedback dos analistas de fraude para melhorar o modelo.
   - Explorar técnicas avançadas como deep learning ou modelos de séries temporais.

O modelo desenvolvido neste projeto oferece uma solução robusta para detecção de fraude em transações de cartão de crédito, com bom equilíbrio entre precisão e recall, e pode ser implementado em um ambiente de produção com os ajustes adequados.